# Joint Method: Continual Learning with QCEdge

## 방법론 개요
Multi-hop 질문에 대해 Query Decomposition과 Knowledge Graph 기반 검색을 결합하여,  
이전 검색 경험(QCEdge)을 활용해 점진적으로 검색 성능을 개선하는 Continual Learning 방식

### 파이프라인 구조
1. **Step 1 (Step 250)**: 초기 검색 + QCEdge 추출
2. **Step 2 (Step 500)**: QCEdge로 그래프 강화 + Intersection QCEdge 계산
3. **Step 3 (Step 750)**: Intersection + Extra QCEdge로 그래프 강화 (`theta_mult=15`, `wub=6`)
4. **Step 4 (Step 1000)**: 더 강한 강화 (`theta_mult=30`, `wub=20`)

In [30]:
# ============================================================
# 1. Imports & Configuration
# ============================================================
"""
필수 라이브러리 및 HippoRAG 모듈 임포트
"""

import os
import json
import re
import math
import random
from copy import deepcopy
from pathlib import Path
from dataclasses import dataclass, field, asdict
from typing import Any, Dict, List, Tuple, Optional
from collections import defaultdict
import numpy as np
import pandas as pd
import sys

# HippoRAG 경로 설정
sys.path.append("/NAS/minyeol/HippoRAG/src")
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

from hipporag.HippoRAG import HippoRAG
from hipporag.llm.openai_gpt import CacheOpenAI
from hipporag.rerank import DSPyFilter
from hipporag.utils.config_utils import BaseConfig
from hipporag.utils.llm_utils import filter_invalid_triples
from hipporag.utils.misc_utils import min_max_normalize, text_processing

In [18]:
# ============================================================
# 2. Configuration & Constants
# ============================================================
"""
실험 설정값 및 상수 정의
- 파일 경로
- Step별 파라미터
- 프롬프트 설정
if __name__ == "__main__":
    STEPS = [250, 500, 750]
    MAX_SAMPLES = 250
    TOP_K_PER_HOP = 5
    TOP_K_PER_BRIDGE = 5
    TOP_K_EDGES = 30
    THETA = 15
    WEIGHT_UPPER_BOUND = 3  # Step 500용 (고정)
    RESET_PROB = 0.85
"""

@dataclass
class PipelineConfig:
    """파이프라인 전체 설정"""
    # 경로 설정
    # 데이터셋 설정 (여러 데이터셋 지원)
    dataset_names: List[str] = field(default_factory=lambda: ["musique"])  # ["musique", "hotpotqa", "2wikimultihopqa"]
    
    # 경로 설정 (dataset_name에 따라 동적으로 설정됨)
    dataset_path: str = None  # dataset_name에 따라 자동 설정
    prompt_root: str = "/NAS/minyeol/HippoRAG/Ours/prompts/QD_bridge2_prompts_reasoning"
    output_dir: str = "/NAS/minyeol/HippoRAG/Ours/outputs/final_method_outputs"
    hippo_base_dir: str = "/NAS/minyeol/HippoRAG/Ours/_hippo_rag_MHQA_CL"  # 데이터셋별 하위 디렉토리 사용
    
    # Step 값
    step_values: List[int] = field(default_factory=lambda: [250, 500, 750, 1000])
    
    # Step 1 파라미터 (Query Decomposition & Initial Retrieval)
    top_k_per_bridge: int = 5       # Bridge question당 검색할 triple 수
    max_bridge_triples: int = 10    # 최대 bridge triple 수
    top_k_per_hop: int = 5         # Sub-question당 검색할 triple 수
    aggregation_method: str = "sum" # Fact 점수 집계: "sum" | "weighted_sum" | "max"
    
    # Step 1 STPPR 파라미터
    top_k_passages_stppr: int = 5  # STPPR 실행할 passage 수
    top_k_edges: int = 30           # 저장할 QCEdge 수
    rbs_alpha: float = 0.5          # RBS backward search reset probability
    rbs_eps: float = 1e-5           # RBS 수렴 threshold
    
    # Step 2 파라미터 (Edge Strengthening with QCEdge)
    theta_step2: float = 15.0       # Edge 강화 강도
    wub_step2: float = 3.0          # Weight upper bound
    
    # Step 3 파라미터 (Intersection + Extra QCEdge) - Step 750
    percentile_step3: float = 50.0  # Extra QCEdge 선택 percentile
    theta_step3: float = 15.0       # 기본 theta
    theta_mult_step3: float = 15.0  # theta multiplier (최종: theta * mult)
    wub_step3: float = 6.0          # Weight upper bound
    
    # Step 4 파라미터 - Step 1000
    percentile_step4: float = 50.0
    theta_step4: float = 15.0
    theta_mult_step4: float = 30.0  # 더 강한 강화
    wub_step4: float = 20.0         # 더 높은 upper bound
    
    # 샘플 수 제한
    max_samples: int = 250

    
    def get_dataset_path(self, dataset_name: str) -> str:
        """데이터셋 이름으로 경로 생성"""
        return f"/NAS/minyeol/HippoRAG/reproduce/dataset/{dataset_name}.json"
    
    def get_hippo_base_dir(self, dataset_name: str) -> str:
        """데이터셋별 hippo base 디렉토리 경로 생성"""
        return f"{self.hippo_base_dir}/{dataset_name}"
    
    def get_output_dir(self, dataset_name: str) -> str:
        """데이터셋별 output 디렉토리 경로 생성"""
        return f"{self.output_dir}/{dataset_name}"


# 프롬프트 파일 매핑
PROMPT_FILES = {
    "atomic": "birdge_extraction_with_description.txt",  # Atomic Bridge Question 추출용
    "context": "simple_query_decomposition.txt",         # Context-aware Query Decomposition용
}

# 전역 설정 인스턴스
CONFIG = PipelineConfig()

print(f"✓ 설정 완료")
print(f"  - Datasets: {CONFIG.dataset_names}")
print(f"  - Output: {CONFIG.output_dir}")
print(f"  - Steps: {CONFIG.step_values}")

✓ 설정 완료
  - Datasets: ['musique']
  - Output: /NAS/minyeol/HippoRAG/Ours/outputs/final_method_outputs
  - Steps: [250, 500, 750, 1000]


In [19]:
# ============================================================
# 3. Data Classes
# ============================================================
"""
파이프라인에서 사용하는 데이터 구조 정의
"""

@dataclass
class AtomicBridgeQuestion:
    """
    Atomic Bridge Question 구조
    
    Multi-hop 질문에서 추출된 단일 factual question과 메타 정보
    
    Attributes:
        question: 추출된 atomic factual question
        description: 해당 질문의 역할 설명 (Role_in_chain)
        info_gain: 이 질문을 해결하면 얻는 정보 (Info_obtained)
        follow_up: 이후 해결해야 할 문제들 (Remaining_subproblems)
    """
    question: str
    description: str
    info_gain: str
    follow_up: List[str]


@dataclass
class HopInfo:
    """
    각 Hop(Sub-question)의 검색 결과 정보
    
    Attributes:
        hop: hop 인덱스 (0-based)
        sub_question: 해당 hop의 sub-question
        top_facts_before_filter: LLM 필터 전 top facts
        top_scores_before_filter: LLM 필터 전 scores
        filtered_facts: LLM 필터 후 facts
        filtered_scores: LLM 필터 후 scores
    """
    hop: int
    sub_question: str
    top_facts_before_filter: List[Tuple[str, str, str]]
    top_scores_before_filter: List[float]
    filtered_facts: List[Tuple[str, str, str]]
    filtered_scores: List[float]


@dataclass
class QCEdgeEntry:
    """
    Query-Conditioned Edge 정보
    
    STPPR을 통해 추출된 query와 연관된 중요 edge
    
    Attributes:
        edge: (source_node_id, target_node_id) 튜플
        importance: 정규화된 중요도 (0~1)
        qc_edge_value: PPR forward score × RBS backward flow
        step: 추출된 step 번호
    """
    edge: Tuple[int, int]
    importance: float
    qc_edge_value: float
    step: str = "unknown"


@dataclass
class RetrievalResult:
    """
    검색 결과 구조
    
    Attributes:
        passages: 검색된 passage 내용 리스트
        scores: 각 passage의 점수
        recall: Recall@K 점수
        hit: Hit@K (1 or 0)
        fallback_used: Dense retrieval fallback 사용 여부
    """
    passages: List[str]
    scores: List[float]
    recall: float
    hit: int
    fallback_used: bool


print("✓ 데이터 클래스 정의 완료")

✓ 데이터 클래스 정의 완료


In [20]:
# ============================================================
# 4. Utility Functions - Data Loading & Saving
# ============================================================
"""
데이터 로딩, 저장, HippoRAG 빌드 등 기본 유틸리티 함수
"""

def load_samples(dataset_name: str, path: str = None) -> List[Dict[str, Any]]:
    """
    데이터셋 로드 (musique, hotpotqa, 2wikimultihopqa 지원)
    
    Args:
        dataset_name: 데이터셋 이름 (musique, hotpotqa, 2wikimultihopqa)
        path: 데이터셋 JSON 파일 경로 (None이면 CONFIG 사용)
    
    Returns:
        샘플 리스트 (각 샘플은 question, context, answer 등 포함)
    """
    path = path or CONFIG.get_dataset_path(dataset_name)
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)


def get_queries(samples: List[Dict[str, Any]]) -> List[str]:
    """샘플에서 질문만 추출"""
    return [sample["question"] for sample in samples]


def get_gold_docs(samples: List[Dict[str, Any]], dataset_name: str = None) -> List[List[str]]:
    """
    샘플에서 정답 문서(supporting facts) 추출 (데이터셋별 처리)
    
    Returns:
        각 샘플별 정답 문서 리스트의 리스트
    """
    gold_docs: List[List[str]] = []
    for sample in samples:
        if 'supporting_facts' in sample:  # hotpotqa, 2wikimultihopqa
            gold_title = set([item[0] for item in sample['supporting_facts']])
            gold_title_and_content_list = [item for item in sample['context'] if item[0] in gold_title]
            if dataset_name and dataset_name.startswith('hotpotqa'):
                gold_doc = [item[0] + '\n' + ''.join(item[1]) for item in gold_title_and_content_list]
            else:
                gold_doc = [item[0] + '\n' + ' '.join(item[1]) for item in gold_title_and_content_list]
        elif 'contexts' in sample:
            gold_doc = [item['title'] + '\n' + item['text'] for item in sample['contexts'] if item['is_supporting']]
        else:
            assert 'paragraphs' in sample, "`paragraphs` should be in sample, or consider the setting not to evaluate retrieval"
            gold_paragraphs = []
            for item in sample['paragraphs']:
                if 'is_supporting' in item and item['is_supporting'] is False:
                    continue
                gold_paragraphs.append(item)
            gold_doc = [item['title'] + '\n' + (item['text'] if 'text' in item else item.get('paragraph_text', '')) for item in gold_paragraphs]
        gold_doc = list(set(gold_doc))
        gold_docs.append(gold_doc)
    return gold_docs

def build_hipporag(step: int, dataset_name: str, config: PipelineConfig = None) -> HippoRAG:
    """
    특정 Step의 HippoRAG 객체 빌드
    
    Args:
        step: step 번호 (250, 500, 750, 1000)
        config: 파이프라인 설정
    
    Returns:
        초기화된 HippoRAG 객체
    """
    config = config or CONFIG
    hippo_base = config.get_hippo_base_dir(dataset_name)
    save_dir = f"{hippo_base}/step_{step}"
    
    cfg = BaseConfig(
        save_dir=save_dir,
        llm_base_url="https://api.openai.com/v1",
        llm_name="gpt-4o-mini",
        dataset="adhoc",
        embedding_model_name="nvidia/NV-Embed-v2",
        force_index_from_scratch=False,
        force_openie_from_scratch=False,
    )
    hippo = HippoRAG(global_config=cfg)
    hippo.prepare_retrieval_objects()
    return hippo


def save_json(data: Any, path: str) -> None:
    """JSON 파일로 저장"""
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    print(f"  → 저장 완료: {path}")


def load_json(path: str) -> Any:
    """JSON 파일 로드"""
    if not os.path.exists(path):
        return None
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)


print("✓ 유틸리티 함수 정의 완료")

✓ 유틸리티 함수 정의 완료


In [21]:
# ============================================================
# 5. LLM Functions - Prompt & Parsing
# ============================================================
"""
LLM 호출 및 응답 파싱 함수
- Atomic Bridge Question 추출
- Query Decomposition
"""

def _load_prompt(name: str) -> str:
    """프롬프트 파일 로드"""
    path = Path(CONFIG.prompt_root) / name
    if not path.is_file():
        raise FileNotFoundError(f"프롬프트 파일 없음: {path}")
    return path.read_text(encoding="utf-8")


def _fill_query_placeholders(template: str, question: str) -> str:
    """프롬프트 템플릿의 플레이스홀더를 실제 질문으로 대체"""
    for placeholder in ("<<ORIGINAL_QUERY>>", "<<YOUR_MULTI_HOP_QUESTION_HERE>>"):
        template = template.replace(placeholder, question.strip())
    return template


def _coerce_followups(value: Any) -> List[str]:
    """follow_up 값을 리스트로 변환"""
    if isinstance(value, list):
        return [str(v).strip() for v in value if str(v).strip()]
    if isinstance(value, str):
        return [seg.strip() for seg in re.split(r"[;\n•\-]+", value) if seg.strip()]
    return []


def _parse_atomic_output(text: str) -> List[AtomicBridgeQuestion]:
    """
    LLM 응답에서 Atomic Bridge Questions 파싱
    
    여러 출력 형식 지원:
    1. {"bridge_questions": [...], "descriptions": [...]} 형식
    2. [{"question": ..., "description": ...}, ...] 형식
    3. 텍스트 형식 (정규식 파싱)
    """
    text = text.strip()
    if not text:
        return []
    
    # JSON 파싱 시도
    try:
        payload = json.loads(text)
    except json.JSONDecodeError:
        payload = None
    
    # 형식 1: bridge_questions + descriptions
    if isinstance(payload, dict) and "bridge_questions" in payload and "descriptions" in payload:
        items = []
        for idx, question in enumerate(payload["bridge_questions"]):
            desc = payload["descriptions"][idx] if idx < len(payload["descriptions"]) else {}
            items.append(AtomicBridgeQuestion(
                question=str(question).strip(),
                description=str(desc.get("Role_in_chain", "")).strip(),
                info_gain=str(desc.get("Info_obtained", "")).strip(),
                follow_up=[str(x).strip() for x in desc.get("Remaining_subproblems", []) if str(x).strip()],
            ))
        return items[:2]  # 최대 2개
    
    # 형식 2: 리스트 형식
    if isinstance(payload, list):
        items = []
        for entry in payload:
            q = entry.get("question") or entry.get("atomic_question")
            if not q:
                continue
            items.append(AtomicBridgeQuestion(
                question=str(q).strip(),
                description=str(entry.get("description", "")).strip(),
                info_gain=str(entry.get("info_gain", "")).strip(),
                follow_up=_coerce_followups(entry.get("follow_up") or entry.get("follow_up_tasks")),
            ))
        if items:
            return items[:2]
    
    # 형식 3: 텍스트 파싱
    pattern = re.compile(
        r"\d+\.\s*Question\s*:\s*(?P<q>.+?)\s*(Description\s*:\s*(?P<d>.+?))?\s*"
        r"(Info\s*Gain\s*:\s*(?P<i>.+?))?\s*(Follow[-\s]*Up\s*:\s*(?P<f>.+?))?(?=\n\d+\.|$)",
        re.IGNORECASE | re.DOTALL,
    )
    items = []
    for match in pattern.finditer(text):
        items.append(AtomicBridgeQuestion(
            question=match.group("q").strip(),
            description=(match.group("d") or "").strip(),
            info_gain=(match.group("i") or "").strip(),
            follow_up=_coerce_followups(match.group("f")),
        ))
    return items[:2]


def extract_atomic_bridge_questions(
    question: str, 
    base_cfg: BaseConfig, 
    max_tokens: int = 512
) -> List[AtomicBridgeQuestion]:
    """
    Step 1.1: Multi-hop 질문에서 Atomic Bridge Questions 추출
    
    LLM을 사용하여 복잡한 multi-hop 질문을 1~2개의 단순한 factual question으로 분해
    각 question에 대해 description, info_gain, follow_up 정보도 함께 추출
    
    Args:
        question: 원본 multi-hop 질문
        base_cfg: HippoRAG BaseConfig
        max_tokens: LLM 응답 최대 토큰
    
    Returns:
        AtomicBridgeQuestion 리스트 (최대 2개)
    """
    prompt = _fill_query_placeholders(_load_prompt(PROMPT_FILES["atomic"]), question)
    
    llm = CacheOpenAI.from_experiment_config(
        BaseConfig(
            save_dir=os.path.join(base_cfg.save_dir, "bridge_atomic"),
            llm_base_url=base_cfg.llm_base_url,
            llm_name=base_cfg.llm_name,
            dataset="adhoc",
            embedding_model_name=base_cfg.embedding_model_name,
        )
    )
    
    raw, *_ = llm.infer(
        messages=[{"role": "user", "content": prompt}],
        max_completion_tokens=max_tokens,
        temperature=0.0
    )
    text = raw[0] if isinstance(raw, (list, tuple)) else raw or ""
    return _parse_atomic_output(text)


def build_context_bundle(
    question: str, 
    atomic_items: List[AtomicBridgeQuestion], 
    triples: List[Tuple[str, str, str]]
) -> str:
    """
    Step 1.3 준비: Query Decomposition을 위한 컨텍스트 번들 생성
    
    Original Query + Atomic Questions + Retrieved Triples를
    LLM에게 전달할 형식으로 조합
    
    Args:
        question: 원본 질문
        atomic_items: 추출된 atomic bridge questions
        triples: 검색된 triples
    
    Returns:
        포맷팅된 컨텍스트 문자열
    """
    # Atomic Questions 섹션
    atomic_section = []
    for idx, item in enumerate(atomic_items, 1):
        follow_up = "; ".join(item.follow_up) if item.follow_up else "N/A"
        atomic_section.append(
            f"{idx}. Question: {item.question}\n"
            f"   Description: {item.description or 'N/A'}\n"
            f"   Info_obtained: {item.info_gain or 'N/A'}\n"
            f"   Remaining_subproblems: {follow_up}"
        )
    
    # Triples 섹션
    triple_section = [f"{idx}. ({s}, {p}, {o})" for idx, (s, p, o) in enumerate(triples, 1)]
    
    return "\n".join([
        "[Original Query]",
        question.strip(),
        "[Atomic Questions]",
        "\n".join(atomic_section) if atomic_section else "No atomic questions extracted.",
        "[Retrieved Triples]",
        "\n".join(triple_section) if triple_section else "No supporting triples retrieved.",
    ])


def decompose_query(
    question: str, 
    context_bundle: str, 
    base_cfg: BaseConfig, 
    max_tokens: int = 512
) -> List[str]:
    """
    Step 1.3: Context-aware Query Decomposition
    
    Original Query + Atomic Questions + Triples 정보를 활용하여
    원본 질문을 최대한 단순한 sub-questions로 분해
    
    Args:
        question: 원본 질문
        context_bundle: build_context_bundle()로 생성된 컨텍스트
        base_cfg: HippoRAG BaseConfig
        max_tokens: LLM 응답 최대 토큰
    
    Returns:
        Sub-questions 리스트
    """
    prompt = _fill_query_placeholders(_load_prompt(PROMPT_FILES["context"]), question)
    prompt = prompt.replace("<<ATOMIC_AND_TRIPLES>>", context_bundle)
    
    llm = CacheOpenAI.from_experiment_config(
        BaseConfig(
            save_dir=os.path.join(base_cfg.save_dir, "bridge_context"),
            llm_base_url=base_cfg.llm_base_url,
            llm_name=base_cfg.llm_name,
            dataset="adhoc",
            embedding_model_name=base_cfg.embedding_model_name,
        )
    )
    
    raw, *_ = llm.infer(
        messages=[{"role": "user", "content": prompt}],
        max_completion_tokens=max_tokens,
        temperature=0.0
    )
    text = raw[0] if isinstance(raw, (list, tuple)) else raw or ""
    text = text.strip()
    
    # JSON 파싱 시도
    try:
        payload = json.loads(text)
        if isinstance(payload, dict):
            payload = payload.get("sub_questions") or payload.get("subqs")
        if isinstance(payload, list):
            return [str(item).strip() for item in payload if str(item).strip()]
    except json.JSONDecodeError:
        pass
    
    # 텍스트 파싱 (번호 형식)
    pattern = re.compile(r"^\s*\d+\.\s*(.+)\s*$")
    lines = [
        pattern.match(line).group(1) if pattern.match(line) else line.strip() 
        for line in text.splitlines()
    ]
    return [line for line in lines if line]


print("✓ LLM 함수 정의 완료")

✓ LLM 함수 정의 완료


In [22]:
# ============================================================
# 6. Triple Retrieval Functions
# ============================================================
"""
Triple(Fact) 검색 관련 함수
- Dense retrieval로 top-k triple 검색
- LLM 기반 triple 필터링
"""

def retrieve_top_facts(
    hippo: HippoRAG, 
    query_text: str, 
    top_k: int = 5
) -> Tuple[List[Tuple[str, str, str]], List[float]]:
    """
    Query와 가장 유사한 top-k facts(triples) 검색
    
    Embedding similarity 기반 dense retrieval
    
    Args:
        hippo: HippoRAG 객체
        query_text: 검색 쿼리
        top_k: 반환할 triple 수
    
    Returns:
        (triples 리스트, scores 리스트) 튜플
    """
    # Query embedding
    query_emb = hippo.embedding_model.batch_encode(
        [query_text], 
        instruction="query_to_fact", 
        norm=True
    )[0]
    
    # Similarity 계산
    sims = hippo.fact_embeddings @ query_emb
    idxs = np.argsort(sims)[::-1][:top_k]
    
    # Fact 정보 추출
    fact_ids = [hippo.fact_node_keys[i] for i in idxs]
    rows = hippo.fact_embedding_store.get_rows(fact_ids)
    facts = [tuple(eval(rows[fid]["content"])) for fid in fact_ids]
    
    return facts, sims[idxs].tolist()


def collect_bridge_triples(
    hippo: HippoRAG, 
    bridge_questions: List[str], 
    top_k_per_bridge: int = None, 
    max_triples: int = None
) -> List[Tuple[str, str, str]]:
    """
    Step 1.2: Bridge Questions로 초기 Triple 검색
    
    각 bridge question에 대해 top-k triples를 검색하고,
    중복 제거 후 max_triples까지 수집
    
    Args:
        hippo: HippoRAG 객체
        bridge_questions: Atomic bridge questions 리스트
        top_k_per_bridge: 각 question당 검색할 triple 수
        max_triples: 최대 수집 triple 수
    
    Returns:
        수집된 unique triples 리스트
    """
    top_k_per_bridge = top_k_per_bridge or CONFIG.top_k_per_bridge
    max_triples = max_triples or CONFIG.max_bridge_triples
    
    gathered, seen = [], set()
    
    for bridge_q in bridge_questions:
        bridge_q = bridge_q.strip()
        if not bridge_q:
            continue
        
        top_facts, _ = retrieve_top_facts(hippo, bridge_q, top_k=top_k_per_bridge)
        normalized = filter_invalid_triples([list(fact) for fact in top_facts])
        
        for triple in normalized:
            triple_tuple = tuple(triple)
            if triple_tuple in seen:
                continue
            seen.add(triple_tuple)
            gathered.append(triple_tuple)
            
            if len(gathered) >= max_triples:
                return gathered
    
    return gathered


def llm_filter_triples(
    hippo: HippoRAG, 
    query: str, 
    candidate_triples: List[Tuple[str, str, str]]
) -> List[Tuple[str, str, str]]:
    """
    LLM을 사용한 Triple 필터링
    
    DSPy Filter를 통해 query와 관련 없는 triples 제거
    
    Args:
        hippo: HippoRAG 객체
        query: 원본/sub 질문
        candidate_triples: 필터링할 후보 triples
    
    Returns:
        필터링된 triples 리스트
    """
    if not candidate_triples:
        return []
    
    filter_obj = DSPyFilter(hippo)
    
    cfg = BaseConfig(
        save_dir=os.path.join(hippo.global_config.save_dir, "bridge_filter"),
        llm_base_url=hippo.global_config.llm_base_url,
        llm_name=hippo.global_config.llm_name,
        dataset="adhoc",
        embedding_model_name=hippo.global_config.embedding_model_name,
    )
    llm = CacheOpenAI.from_experiment_config(cfg)
    
    # 필터 요청
    payload = {"fact": [list(triple) for triple in candidate_triples]}
    messages = deepcopy(filter_obj.message_template)
    messages.append({
        "role": "user", 
        "content": filter_obj.one_input_template.format(
            question=query, 
            fact_before_filter=json.dumps(payload)
        )
    })
    
    raw, *_ = llm.infer(messages=messages, max_completion_tokens=512, temperature=0.0)
    text = raw[0] if isinstance(raw, (list, tuple)) else raw or ""
    
    # 파싱 및 매칭
    generated = filter_obj.parse_filter(text)
    matched, cand_strings = [], [str(t) for t in candidate_triples]
    
    for fact in generated:
        if isinstance(fact, list) and len(fact) == 3:
            triple = tuple(str(x) for x in fact)
            try:
                idx = cand_strings.index(str(triple))
                if candidate_triples[idx] not in matched:
                    matched.append(candidate_triples[idx])
            except ValueError:
                continue
    
    return matched


def collect_sub_question_triples(
    hippo: HippoRAG, 
    sub_questions: List[str], 
    top_k: int = None
) -> List[Dict[str, Any]]:
    """
    Step 1.4: Sub-Questions로 Triple 검색 + LLM 필터링
    
    각 sub-question(hop)에 대해:
    1. Dense retrieval로 top-k triples 검색
    2. LLM filter로 관련성 낮은 triples 제거
    
    Args:
        hippo: HippoRAG 객체
        sub_questions: 분해된 sub-questions
        top_k: 각 question당 검색할 triple 수
    
    Returns:
        각 hop의 정보를 담은 딕셔너리 리스트
    """
    top_k = top_k or CONFIG.top_k_per_hop
    hop_infos = []
    
    for hop_idx, sub_q in enumerate(sub_questions):
        sub_q = sub_q.strip()
        if not sub_q:
            continue
        
        # Dense retrieval
        top_facts, top_scores = retrieve_top_facts(hippo, sub_q, top_k=top_k)
        
        # LLM filtering
        filtered = llm_filter_triples(hippo, sub_q, top_facts)
        
        # 필터링된 facts의 scores 매핑
        lookup = dict(zip(top_facts, top_scores))
        
        hop_infos.append({
            "hop": hop_idx,
            "sub_question": sub_q,
            "top_facts_before_filter": top_facts,
            "top_scores_before_filter": top_scores,
            "filtered_facts": filtered,
            "filtered_scores": [lookup.get(fact, 0.0) for fact in filtered],
        })
    
    return hop_infos


print("✓ Triple 검색 함수 정의 완료")

✓ Triple 검색 함수 정의 완료


In [23]:
# ============================================================
# 7. Seed Vector & Index Mapping Functions
# ============================================================
"""
Fact Seed Vector 생성 및 Triple-Index 매핑 함수
"""

def _map_triples_to_indices(
    hippo: HippoRAG, 
    triples: List[Tuple[str, str, str]]
) -> List[int]:
    """
    Triple을 HippoRAG의 fact embedding index로 매핑
    
    Args:
        hippo: HippoRAG 객체
        triples: 매핑할 triples
    
    Returns:
        각 triple에 대응하는 fact index 리스트
    """
    if not triples:
        return []
    
    # Fact ID -> Row 매핑 생성
    id_to_row = hippo.fact_embedding_store.get_rows(hippo.fact_node_keys)
    
    # Triple 문자열 -> Index 룩업 테이블
    lookup = {}
    for idx, fid in enumerate(hippo.fact_node_keys):
        triple = tuple(eval(id_to_row[fid]["content"]))
        lookup[str(tuple(text_processing(list(triple))))] = idx
    
    # 매핑
    indices = []
    for triple in triples:
        key = str(tuple(text_processing(list(triple))))
        if key in lookup:
            indices.append(lookup[key])
    
    return indices


def build_fact_seed_vector(
    hop_infos: List[Dict[str, Any]], 
    hippo: HippoRAG, 
    aggregation_method: str = None
) -> Tuple[np.ndarray, List[int], List[Tuple[str, str, str]]]:
    """
    Step 1.5: Fact Seed Vector 생성
    
    각 hop의 filtered_facts와 scores를 집계하여
    PPR의 seed로 사용할 fact score vector 생성
    
    Args:
        hop_infos: collect_sub_question_triples()의 결과
        hippo: HippoRAG 객체
        aggregation_method: 점수 집계 방식
            - "sum": 점수 합산
            - "weighted_sum": hop 빈도 가중치 적용
            - "max": 최대값 사용
    
    Returns:
        (fact_scores_vec, seed_indices, seed_facts) 튜플
    """
    aggregation_method = aggregation_method or CONFIG.aggregation_method
    
    fact_scores = np.zeros(len(hippo.fact_embeddings))
    fact_seed_map, fact_hop_count = {}, {}
    
    for hop in hop_infos:
        for fact, score in zip(hop.get("filtered_facts") or [], hop.get("filtered_scores") or []):
            idxs = _map_triples_to_indices(hippo, [fact])
            if not idxs:
                continue
            
            idx = idxs[0]
            
            if idx not in fact_seed_map:
                fact_seed_map[idx] = fact
                fact_hop_count[idx] = 0
            fact_hop_count[idx] += 1
            
            # 집계 방식에 따른 점수 계산
            if aggregation_method == "sum":
                fact_scores[idx] += float(score)
            elif aggregation_method == "weighted_sum":
                fact_scores[idx] += float(score) * (1.0 + fact_hop_count[idx] * 0.1)
            else:  # max
                fact_scores[idx] = max(fact_scores[idx], float(score))
    
    # 정규화
    if fact_scores.sum() > 0:
        fact_scores = min_max_normalize(fact_scores)
    
    seed_indices = list(fact_seed_map.keys())
    seed_facts = [fact_seed_map[idx] for idx in seed_indices]
    
    return fact_scores, seed_indices, seed_facts


def build_seed_vector_from_stored(
    hippo: HippoRAG, 
    stored_triples: List[Tuple[str, str, str]], 
    stored_scores: List[float]
) -> Tuple[np.ndarray, List[int], List[Tuple[str, str, str]]]:
    """
    저장된 Triple과 Score로부터 Seed Vector 재생성
    
    Step 2, 3, 4에서 Memory Bank의 데이터를 사용하여 seed vector 생성
    
    Args:
        hippo: HippoRAG 객체
        stored_triples: 저장된 triples
        stored_scores: 각 triple의 점수
    
    Returns:
        (fact_scores_vec, seed_indices, seed_facts) 튜플
    """
    seed_indices = _map_triples_to_indices(hippo, stored_triples)
    seed_facts = stored_triples[:len(seed_indices)]
    scores_for_indices = stored_scores[:len(seed_indices)]
    
    fact_scores_vec = np.zeros(len(hippo.fact_embeddings))
    for fact_idx, score in zip(seed_indices, scores_for_indices):
        fact_scores_vec[fact_idx] = float(score)
    
    if fact_scores_vec.sum() > 0:
        fact_scores_vec = min_max_normalize(fact_scores_vec)
    
    return fact_scores_vec, seed_indices, seed_facts


print("✓ Seed Vector 함수 정의 완료")

✓ Seed Vector 함수 정의 완료


In [24]:
# ============================================================
# 8. STPPR & QCEdge Extraction Functions
# ============================================================
"""
STPPR (Source-Target PPR) 및 QCEdge 추출 함수
- RBS (Reverse Backward Search) 알고리즘
- Query-Conditioned Edge 계산
"""

def run_rbs_backward_search(
    graph,
    target_node: int,
    alpha: float = None,
    eps: float = None,
    error_type: int = 1
) -> Tuple[Dict[int, float], Dict[Tuple[int, int], float]]:
    """
    RBS (Reverse Backward Search) 알고리즘
    
    Target 노드로부터 역방향으로 PPR 스코어를 전파하여
    각 노드의 기여도와 edge flow를 계산
    
    이를 통해 특정 passage(target)에 대한 각 edge의 중요도 파악 가능
    
    Args:
        graph: igraph Graph 객체
        target_node: 역방향 탐색 시작 노드 (passage node)
        alpha: Reset probability (높을수록 target 근처에 집중)
        eps: 수렴 threshold
        error_type: 에러 계산 방식 (1 또는 2)
    
    Returns:
        (final_reserve, edge_flow) 튜플
        - final_reserve: 각 노드의 최종 reserve 값
        - edge_flow: 각 edge (u, v)의 flow 값
    """
    alpha = alpha or CONFIG.rbs_alpha
    eps = eps or CONFIG.rbs_eps
    
    n = graph.vcount()
    residue = [defaultdict(float), defaultdict(float)]
    final_reserve = defaultdict(float)
    edge_flow = defaultdict(float)
    visited = set()
    candidate_set = [[], []]
    candidate_count = [0, 0]
    
    # 초기화
    level = 0
    residue[0][target_node] = 1.0
    candidate_set[0] = [target_node]
    candidate_count[0] = 1
    
    # 최대 레벨 계산
    L = int(math.ceil(math.log(eps) / math.log(1 - alpha))) + 1 if alpha < 1.0 else 1
    
    while level <= L:
        current_level = level % 2
        next_level = (level + 1) % 2
        
        candidate_cnt = candidate_count[current_level]
        if candidate_cnt == 0:
            break
        
        candidate_count[current_level] = 0
        
        for j in range(candidate_cnt):
            temp_node = candidate_set[current_level][j]
            temp_r = residue[current_level][temp_node]
            residue[current_level][temp_node] = 0.0
            
            if temp_node not in visited:
                visited.add(temp_node)
            
            # Reserve 업데이트
            final_reserve[temp_node] += alpha * temp_r
            
            if level == L:
                continue
            
            # 이웃 노드로 전파
            neighbors = graph.neighbors(temp_node, mode='all')
            
            for neighbor in neighbors:
                degree = graph.degree(neighbor, mode='all')
                if degree == 0:
                    continue
                
                incre = temp_r * (1 - alpha) / float(degree)
                ran = random.random()
                
                # Edge flow 및 residue 업데이트
                if error_type == 1:
                    if math.sqrt(degree) * eps <= (1 - alpha) * temp_r:
                        edge_flow[(neighbor, temp_node)] += incre
                        residue[next_level][neighbor] += incre
                    else:
                        if math.sqrt(degree) * ran * eps <= (1 - alpha) * temp_r:
                            ran_incre = eps / math.sqrt(degree)
                            edge_flow[(neighbor, temp_node)] += ran_incre
                            residue[next_level][neighbor] += ran_incre
                        else:
                            break
                else:
                    if degree * eps <= (1 - alpha) * temp_r:
                        edge_flow[(neighbor, temp_node)] += incre
                        residue[next_level][neighbor] += incre
                    else:
                        if degree * eps * ran <= (1 - alpha) * temp_r:
                            ran_incre = eps
                            edge_flow[(neighbor, temp_node)] += ran_incre
                            residue[next_level][neighbor] += ran_incre
                        else:
                            break
                
                # 다음 레벨 후보 추가
                if residue[next_level][neighbor] > eps and neighbor not in candidate_set[next_level]:
                    candidate_set[next_level].append(neighbor)
                    candidate_count[next_level] += 1
        
        level += 1
    
    return dict(final_reserve), dict(edge_flow)


def extract_qc_edges(
    hippo: HippoRAG,
    question: str,
    fact_scores_vec: np.ndarray,
    seed_indices: List[int],
    seed_facts: List[Tuple[str, str, str]],
    top_k_passages: int = None,
    top_k_edges: int = None,
    eps: float = 1e-6,
    step_label: str = "unknown"
) -> List[Dict[str, Any]]:
    """
    Step 1.7 / 2.4 / 3.x: QCEdge (Query-Conditioned Edge) 추출
    
    PPR로 top-k passages를 검색한 후,
    각 passage에 대해 RBS backward search를 수행하여
    query와 연관된 중요 edges(QCEdge)를 추출
    
    QCEdge 값 = PPR forward score × RBS backward flow
    
    Args:
        hippo: HippoRAG 객체
        question: 원본 질문
        fact_scores_vec: Seed fact scores
        seed_indices: Seed fact indices
        seed_facts: Seed facts
        top_k_passages: STPPR 실행할 passage 수
        top_k_edges: 저장할 QCEdge 수
        eps: importance 계산 시 epsilon
        step_label: step 라벨
    
    Returns:
        QCEdge 정보 딕셔너리 리스트
    """
    top_k_passages = top_k_passages or CONFIG.top_k_passages_stppr
    top_k_edges = top_k_edges or CONFIG.top_k_edges
    
    if fact_scores_vec.sum() == 0 or not seed_indices:
        return []
    
    # PPR 실행하여 top-k passages 검색
    doc_ids, doc_scores, _, _ = hippo.graph_search_with_fact_entities(
        query=question,
        link_top_k=hippo.global_config.linking_top_k,
        query_fact_scores=fact_scores_vec,
        top_k_facts=seed_facts,
        top_k_fact_indices=seed_indices,
        passage_node_weight=hippo.global_config.passage_node_weight,
    )
    
    query_conditioned_edges = defaultdict(float)
    
    # 각 top-k passage에 대해 RBS 실행 (STPPR)
    for passage_idx in doc_ids[:top_k_passages]:
        passage_node_id = hippo.passage_node_idxs[passage_idx]
        
        _, edge_flow = run_rbs_backward_search(
            graph=hippo.graph,
            target_node=passage_node_id,
        )
        
        # Forward score와 backward flow를 결합
        forward_score = doc_scores[list(doc_ids).index(passage_idx)]
        for (u, v), flow in edge_flow.items():
            qc_edge_value = forward_score * flow
            query_conditioned_edges[(u, v)] += qc_edge_value
    
    if len(query_conditioned_edges) == 0:
        return []
    
    # Top-k edges 선택
    sorted_qc_edges = sorted(query_conditioned_edges.items(), key=lambda x: x[1], reverse=True)
    max_qc_edge = max(query_conditioned_edges.values())
    top_k_sorted = sorted_qc_edges[:top_k_edges]
    
    # 결과 구성
    memory_bank_entries = []
    for (u, v), qc_edge_value in top_k_sorted:
        importance = (qc_edge_value / max_qc_edge) + eps
        memory_bank_entries.append({
            'step': step_label,
            'edge': [int(u), int(v)],
            'importance': float(importance),
            'qc_edge_value': float(qc_edge_value)
        })
    
    return memory_bank_entries


print("✓ STPPR & QCEdge 함수 정의 완료")

✓ STPPR & QCEdge 함수 정의 완료


In [25]:
# ============================================================
# 9. Graph Strengthening Functions
# ============================================================
"""
QCEdge를 활용한 그래프 강화 함수
- Step 2: 단순 QCEdge 강화
- Step 3, 4: Intersection + Extra QCEdge 강화
"""

def strengthen_edges_with_qc_edges(
    graph,
    qc_edges: List[Dict[str, Any]],
    theta: float = None,
    weight_upper_bound: float = None
):
    """
    Step 2: QCEdge로 그래프 엣지 가중치 강화
    
    수식: new_weight = current_weight × (1.0 + √(importance × theta))
    
    Args:
        graph: igraph Graph 객체 (원본)
        qc_edges: QCEdge 정보 리스트
        theta: 강화 강도 (높을수록 더 강한 강화)
        weight_upper_bound: 가중치 상한
    
    Returns:
        강화된 그래프 복사본
    """
    theta = theta or CONFIG.theta_step2
    weight_upper_bound = weight_upper_bound or CONFIG.wub_step2
    
    strengthened_graph = graph.copy()
    
    # Edge -> Importance 매핑
    edges_to_strengthen = {
        tuple(entry['edge']): entry['importance'] 
        for entry in qc_edges
    }
    
    for edge in strengthened_graph.es:
        u, v = edge.source, edge.target
        edge_key = (u, v)
        reverse_key = (v, u)
        
        # 강화 대상 엣지인지 확인
        if edge_key in edges_to_strengthen:
            importance = edges_to_strengthen[edge_key]
        elif reverse_key in edges_to_strengthen:
            importance = edges_to_strengthen[reverse_key]
        else:
            continue
        
        # 가중치 강화
        current_weight = edge['weight'] if 'weight' in edge.attributes() else 1.0
        new_weight = current_weight * (1.0 + np.sqrt(importance * theta))
        new_weight = min(new_weight, weight_upper_bound)
        edge['weight'] = new_weight
    
    return strengthened_graph


def compute_qc_edge_intersection(
    qc_edges_step1: List[Dict[str, Any]],
    qc_edges_step2: List[Dict[str, Any]]
) -> List[Dict[str, Any]]:
    """
    Step 2.5: 두 단계의 QCEdge 교집합 계산
    
    두 단계 모두에서 중요했던 edges만 선택
    (신뢰도 높은 edges)
    
    Args:
        qc_edges_step1: Step 1에서 추출한 QCEdges
        qc_edges_step2: Step 2에서 추출한 QCEdges
    
    Returns:
        교집합 QCEdge 리스트 (importance = 두 단계 중 최소값)
    """
    edges_step1 = {tuple(entry['edge']): entry for entry in qc_edges_step1}
    edges_step2 = {tuple(entry['edge']): entry for entry in qc_edges_step2}
    
    intersection_edges = []
    
    for edge_key, entry1 in edges_step1.items():
        reverse_key = (edge_key[1], edge_key[0])
        
        if edge_key in edges_step2:
            entry2 = edges_step2[edge_key]
        elif reverse_key in edges_step2:
            entry2 = edges_step2[reverse_key]
        else:
            continue
        
        # 교집합 엣지: 두 단계 중 최소 importance 사용
        intersection_edges.append({
            'edge': list(edge_key),
            'importance': min(entry1.get('importance', 0), entry2.get('importance', 0)),
            'qc_edge_value': min(entry1.get('qc_edge_value', 0), entry2.get('qc_edge_value', 0)),
            'step1_importance': entry1.get('importance', 0),
            'step2_importance': entry2.get('importance', 0),
        })
    
    return intersection_edges


def select_extra_qc_edges_by_percentile(
    qc_edges_step1: List[Dict[str, Any]],
    percentile: float = None
) -> List[Dict[str, Any]]:
    """
    Step 3.1: Percentile 기준으로 Extra QCEdge 선택
    
    Step 1의 QCEdge 중 상위 percentile에 해당하는 edges 선택
    
    Args:
        qc_edges_step1: Step 1에서 추출한 QCEdges
        percentile: 선택할 상위 percentile (기본: 50)
    
    Returns:
        선택된 Extra QCEdge 리스트
    """
    percentile = percentile or CONFIG.percentile_step3
    
    if not qc_edges_step1:
        return []
    
    # qc_edge_value 기준 정렬
    sorted_edges = sorted(qc_edges_step1, key=lambda x: x.get('qc_edge_value', 0), reverse=True)
    
    # Percentile threshold 계산
    values = [e.get('qc_edge_value', 0) for e in sorted_edges]
    if not values:
        return []
    
    threshold = np.percentile(values, percentile)
    
    # Threshold 이상인 edges 선택
    extra_edges = [e for e in sorted_edges if e.get('qc_edge_value', 0) >= threshold]
    
    return extra_edges


def strengthen_edges_with_intersection_and_extra(
    graph,
    intersection_edges: List[Dict[str, Any]],
    extra_edges: List[Dict[str, Any]],
    theta: float = None,
    theta_multiplier: float = None,
    weight_upper_bound: float = None
):
    """
    Step 3, 4: Intersection QCEdge + Extra QCEdge로 그래프 강화
    
    effective_theta = theta × theta_multiplier
    수식: new_weight = current_weight × (1.0 + √(importance × effective_theta))
    
    Args:
        graph: igraph Graph 객체 (원본)
        intersection_edges: 교집합 QCEdges
        extra_edges: Extra QCEdges
        theta: 기본 theta
        theta_multiplier: theta 배수
        weight_upper_bound: 가중치 상한
    
    Returns:
        강화된 그래프 복사본
    """
    theta = theta or CONFIG.theta_step3
    theta_multiplier = theta_multiplier or CONFIG.theta_mult_step3
    weight_upper_bound = weight_upper_bound or CONFIG.wub_step3
    
    strengthened_graph = graph.copy()
    effective_theta = theta * theta_multiplier
    
    # Edge -> Entry 매핑
    intersection_dict = {tuple(entry['edge']): entry for entry in intersection_edges}
    extra_dict = {tuple(entry['edge']): entry for entry in extra_edges}
    
    for edge in strengthened_graph.es:
        u, v = edge.source, edge.target
        edge_key = (u, v)
        reverse_key = (v, u)
        
        current_weight = edge['weight'] if 'weight' in edge.attributes() else 1.0
        
        # 교집합 엣지인 경우
        if edge_key in intersection_dict or reverse_key in intersection_dict:
            entry = intersection_dict.get(edge_key) or intersection_dict.get(reverse_key)
            importance = entry.get('importance', 0)
            new_weight = current_weight * (1.0 + np.sqrt(importance * effective_theta))
            new_weight = min(new_weight, weight_upper_bound)
            edge['weight'] = new_weight
        # Extra 엣지인 경우
        elif edge_key in extra_dict or reverse_key in extra_dict:
            entry = extra_dict.get(edge_key) or extra_dict.get(reverse_key)
            importance = entry.get('importance', 0)
            new_weight = current_weight * (1.0 + np.sqrt(importance * effective_theta))
            new_weight = min(new_weight, weight_upper_bound)
            edge['weight'] = new_weight
    
    return strengthened_graph


print("✓ Graph Strengthening 함수 정의 완료")

✓ Graph Strengthening 함수 정의 완료


In [26]:
# ============================================================
# 10. Graph Search & Evaluation Functions
# ============================================================
"""
PPR 기반 검색 및 성능 평가 함수
"""

def run_graph_search(
    hippo: HippoRAG,
    question: str,
    gold: List[str],
    fact_scores_vec: np.ndarray,
    seed_indices: List[int],
    seed_facts: List[Tuple[str, str, str]],
    top_k: int = 5
) -> RetrievalResult:
    """
    PPR 기반 Passage 검색 및 평가
    
    Seed vector가 비어있으면 Dense Retrieval로 fallback
    
    Args:
        hippo: HippoRAG 객체
        question: 검색 질문
        gold: 정답 문서 리스트
        fact_scores_vec: Seed fact scores
        seed_indices: Seed fact indices
        seed_facts: Seed facts
        top_k: 반환할 passage 수
    
    Returns:
        RetrievalResult 객체
    """
    fallback = False
    
    if fact_scores_vec.sum() == 0 or not seed_indices:
        # Dense Retrieval Fallback
        doc_ids, doc_scores = hippo.dense_passage_retrieval(question)
        fallback = True
    else:
        # PPR Graph Search
        doc_ids, doc_scores, _, _ = hippo.graph_search_with_fact_entities(
            query=question,
            link_top_k=hippo.global_config.linking_top_k,
            query_fact_scores=fact_scores_vec,
            top_k_facts=seed_facts,
            top_k_fact_indices=seed_indices,
            passage_node_weight=hippo.global_config.passage_node_weight,
        )
    
    # Top-k passages 추출
    top_passages = [
        hippo.chunk_embedding_store.get_row(hippo.passage_node_keys[idx])["content"] 
        for idx in doc_ids[:top_k]
    ]
    top_scores = doc_scores[:top_k].tolist()
    
    # 평가
    retrieved_set = {p.strip() for p in top_passages}
    gold_set = {g.strip() for g in gold}
    
    if gold_set:
        overlap = retrieved_set & gold_set
        recall = len(overlap) / len(gold_set)
        hit = int(bool(overlap))
    else:
        recall, hit = 0.0, 0
    
    return RetrievalResult(
        passages=top_passages,
        scores=top_scores,
        recall=recall,
        hit=hit,
        fallback_used=fallback
    )


def run_fallback_search(
    hippo: HippoRAG,
    question: str,
    gold: List[str],
    top_k: int = 5
) -> RetrievalResult:
    """
    Dense Retrieval Fallback 검색
    
    Memory bank에 데이터가 없는 경우 사용
    
    Args:
        hippo: HippoRAG 객체
        question: 검색 질문
        gold: 정답 문서 리스트
        top_k: 반환할 passage 수
    
    Returns:
        RetrievalResult 객체
    """
    doc_ids, doc_scores = hippo.dense_passage_retrieval(question)
    
    top_passages = [
        hippo.chunk_embedding_store.get_row(hippo.passage_node_keys[idx])["content"] 
        for idx in doc_ids[:top_k]
    ]
    top_scores = doc_scores[:top_k].tolist()
    
    retrieved_set = {p.strip() for p in top_passages}
    gold_set = {g.strip() for g in gold}
    
    if gold_set:
        overlap = retrieved_set & gold_set
        recall = len(overlap) / len(gold_set)
        hit = int(bool(overlap))
    else:
        recall, hit = 0.0, 0
    
    return RetrievalResult(
        passages=top_passages,
        scores=top_scores,
        recall=recall,
        hit=hit,
        fallback_used=True
    )


print("✓ 검색 & 평가 함수 정의 완료")

✓ 검색 & 평가 함수 정의 완료


In [27]:
# ============================================================
# 11. Step Execution Functions
# ============================================================
"""
각 Step별 파이프라인 실행 함수
- Step 1 (250): 초기 검색 + QCEdge 추출
- Step 2 (500): QCEdge 강화 + Intersection 계산
- Step 3 (750): Intersection + Extra 강화
- Step 4 (1000): 더 강한 강화
"""

def run_step1(
    hippo: HippoRAG,
    queries: List[str],
    gold_docs: List[List[str]],
    config: PipelineConfig = None,
    verbose: bool = True
) -> Tuple[Dict[str, Any], Dict[str, Any], List[Dict[str, Any]], Dict[str, float]]:
    """
    Step 1 (Step 250): 초기 검색 및 QCEdge 추출
    
    파이프라인:
    1. Atomic Bridge Question 추출
    2. Bridge Triple 검색
    3. Context-aware Query Decomposition
    4. Sub-Question Triple 검색 + LLM 필터링
    5. Fact Seed Vector 생성
    6. PPR → Passage 검색
    7. STPPR → QCEdge 추출
    
    Returns:
        (memory_bank, buffer, log_records, metrics) 튜플
    """
    config = config or CONFIG
    memory_bank = {}
    buffer = {}
    log_records = []
    recalls, hits = [], []
    
    for idx, question in enumerate(queries):
        if verbose and ((idx + 1) % 10 == 0 or (idx + 1) == len(queries)):
            print(f"  [Step 250] {idx + 1}/{len(queries)} ({100 * (idx + 1) / len(queries):.1f}%)")
        
        # 1. Atomic Bridge Question 추출
        atomic_items = extract_atomic_bridge_questions(question, hippo.global_config)
        bridge_questions = [item.question for item in atomic_items] or [question]
        
        # 2. Bridge Triple 검색
        bridge_triples = collect_bridge_triples(hippo, bridge_questions)
        
        # 3. Query Decomposition
        context_bundle = build_context_bundle(question, atomic_items, bridge_triples)
        sub_questions = decompose_query(question, context_bundle, hippo.global_config) or bridge_questions
        
        # 4. Sub-Question Triple 검색 + 필터링
        hop_infos = collect_sub_question_triples(hippo, sub_questions)
        
        # 5. Seed Vector 생성
        fact_scores_vec, seed_indices, seed_facts = build_fact_seed_vector(hop_infos, hippo)
        
        # Triple + Score 저장용
        triple_to_score = {}
        for hop_info in hop_infos:
            for fact, score in zip(hop_info.get("filtered_facts", []), hop_info.get("filtered_scores", [])):
                fact_tuple = tuple(fact)
                if fact_tuple not in triple_to_score:
                    triple_to_score[fact_tuple] = float(score)
                else:
                    triple_to_score[fact_tuple] = max(triple_to_score[fact_tuple], float(score))
        
        stored_triples_with_scores = [
            {"triple": list(triple), "score": score} 
            for triple, score in triple_to_score.items()
        ]
        
        # 6. PPR → Passage
        result = run_graph_search(
            hippo=hippo,
            question=question,
            gold=gold_docs[idx],
            fact_scores_vec=fact_scores_vec,
            seed_indices=seed_indices,
            seed_facts=seed_facts,
        )
        
        # 7. STPPR → QCEdge 추출
        qc_edges = extract_qc_edges(
            hippo=hippo,
            question=question,
            fact_scores_vec=fact_scores_vec,
            seed_indices=seed_indices,
            seed_facts=seed_facts,
            step_label="250"
        )
        
        # Memory Bank 저장
        memory_bank[question] = {
            "retrieved_triples_with_scores": stored_triples_with_scores,
        }
        
        # Buffer 저장
        buffer[question] = {
            "qc_edges_step1": qc_edges,
        }
        
        # 로그 기록
        log_records.append({
            "idx": idx,
            "question": question,
            "atomic_bridge_questions": [asdict(item) for item in atomic_items],
            "bridge_questions": bridge_questions,
            "bridge_triples": [list(t) for t in bridge_triples],
            "sub_questions": sub_questions,
            "seed_facts": [list(t) for t in seed_facts],
            "qc_edges": qc_edges,
            "retrieved_passages_top5": result.passages,
            "retrieved_scores_top5": result.scores,
            "gold_docs": gold_docs[idx],
            "recall@5": result.recall,
            "hit@5": result.hit,
            "fallback_used": result.fallback_used,
        })
        
        recalls.append(result.recall)
        hits.append(result.hit)
    
    metrics = {
        "mean_recall@5": float(np.mean(recalls)) if recalls else 0.0,
        "hit_rate@5": float(np.mean(hits)) if hits else 0.0,
    }
    
    return memory_bank, buffer, log_records, metrics


def run_step2(
    hippo: HippoRAG,
    queries: List[str],
    gold_docs: List[List[str]],
    memory_bank: Dict[str, Any],
    buffer: Dict[str, Any],
    config: PipelineConfig = None,
    verbose: bool = True
) -> Tuple[Dict[str, Any], List[Dict[str, Any]], Dict[str, float]]:
    """
    Step 2 (Step 500): QCEdge로 그래프 강화 + Intersection 계산
    
    파이프라인:
    1. Step 1의 QCEdge로 그래프 강화
    2. 저장된 Triple로 Seed Vector 재생성
    3. PPR → Passage 검색
    4. STPPR → QCEdge 추출 (Step 2)
    5. Intersection QCEdge 계산
    
    Returns:
        (updated_buffer, log_records, metrics) 튜플
    """
    config = config or CONFIG
    original_graph = hippo.graph.copy()
    log_records = []
    recalls, hits = [], []
    
    for idx, question in enumerate(queries):
        if verbose and ((idx + 1) % 10 == 0 or (idx + 1) == len(queries)):
            print(f"  [Step 500] {idx + 1}/{len(queries)} ({100 * (idx + 1) / len(queries):.1f}%)")
        
        if question not in memory_bank or question not in buffer:
            result = run_fallback_search(hippo, question, gold_docs[idx])
            log_records.append({
                "idx": idx,
                "question": question,
                "recall@5": result.recall,
                "hit@5": result.hit,
                "fallback_used": True,
            })
            recalls.append(result.recall)
            hits.append(result.hit)
            continue
        
        mem_data = memory_bank[question]
        buffer_data = buffer[question]
        qc_edges_step1 = buffer_data.get("qc_edges_step1", [])
        
        # 1. QCEdge로 그래프 강화
        if qc_edges_step1:
            hippo.graph = strengthen_edges_with_qc_edges(
                original_graph,
                qc_edges_step1,
                theta=config.theta_step2,
                weight_upper_bound=config.wub_step2
            )
        
        # 2. 저장된 Triple로 Seed Vector 재생성
        stored_data = mem_data.get("retrieved_triples_with_scores", [])
        
        if stored_data:
            stored_triples = [tuple(item["triple"]) for item in stored_data]
            stored_scores = [item["score"] for item in stored_data]
            
            fact_scores_vec, seed_indices, seed_facts = build_seed_vector_from_stored(
                hippo, stored_triples, stored_scores
            )
            
            # 3. PPR → Passage
            result = run_graph_search(
                hippo=hippo,
                question=question,
                gold=gold_docs[idx],
                fact_scores_vec=fact_scores_vec,
                seed_indices=seed_indices,
                seed_facts=seed_facts,
            )
            
            # 4. STPPR → QCEdge 추출 (Step 2)
            qc_edges_step2 = extract_qc_edges(
                hippo=hippo,
                question=question,
                fact_scores_vec=fact_scores_vec,
                seed_indices=seed_indices,
                seed_facts=seed_facts,
                step_label="500"
            )
            
            # 5. Intersection 계산
            intersection_qc_edges = compute_qc_edge_intersection(qc_edges_step1, qc_edges_step2)
            
            # Buffer 업데이트
            buffer[question]["qc_edges_step2"] = qc_edges_step2
            buffer[question]["intersection_qc_edges"] = intersection_qc_edges
            
            log_records.append({
                "idx": idx,
                "question": question,
                "qc_edges_step1": qc_edges_step1,
                "qc_edges_step2": qc_edges_step2,
                "intersection_qc_edges": intersection_qc_edges,
                "retrieved_passages_top5": result.passages,
                "retrieved_scores_top5": result.scores,
                "gold_docs": gold_docs[idx],
                "recall@5": result.recall,
                "hit@5": result.hit,
                "fallback_used": result.fallback_used,
            })
        else:
            result = run_fallback_search(hippo, question, gold_docs[idx])
            log_records.append({
                "idx": idx,
                "question": question,
                "recall@5": result.recall,
                "hit@5": result.hit,
                "fallback_used": True,
            })
        
        recalls.append(result.recall)
        hits.append(result.hit)
        
        # 그래프 복원
        hippo.graph = original_graph
    
    metrics = {
        "mean_recall@5": float(np.mean(recalls)) if recalls else 0.0,
        "hit_rate@5": float(np.mean(hits)) if hits else 0.0,
    }
    
    return buffer, log_records, metrics


def run_step3_or_4(
    hippo: HippoRAG,
    queries: List[str],
    gold_docs: List[List[str]],
    memory_bank: Dict[str, Any],
    buffer: Dict[str, Any],
    step_num: int,
    theta_multiplier: float,
    weight_upper_bound: float,
    percentile: float = None,
    config: PipelineConfig = None,
    verbose: bool = True
) -> Tuple[List[Dict[str, Any]], Dict[str, float]]:
    """
    Step 3 (750) / Step 4 (1000): Intersection + Extra QCEdge로 그래프 강화
    
    파이프라인:
    1. Extra QCEdge 선택 (percentile 기준)
    2. Intersection + Extra로 그래프 강화
    3. PPR → Passage 검색
    
    Args:
        hippo: HippoRAG 객체
        queries: 질문 리스트
        gold_docs: 정답 문서 리스트
        memory_bank: Step 1에서 생성된 memory bank
        buffer: QCEdge 정보 버퍼
        step_num: step 번호 (750 또는 1000)
        theta_multiplier: theta 배수
        weight_upper_bound: 가중치 상한
        percentile: Extra QCEdge 선택 percentile
        config: 파이프라인 설정
        verbose: 진행 상황 출력 여부
    
    Returns:
        (log_records, metrics) 튜플
    """
    config = config or CONFIG
    percentile = percentile or config.percentile_step3
    
    original_graph = hippo.graph.copy()
    log_records = []
    recalls, hits = [], []
    
    for idx, question in enumerate(queries):
        if verbose and ((idx + 1) % 10 == 0 or (idx + 1) == len(queries)):
            print(f"  [Step {step_num}] {idx + 1}/{len(queries)} ({100 * (idx + 1) / len(queries):.1f}%)")
        
        if question not in memory_bank or question not in buffer:
            result = run_fallback_search(hippo, question, gold_docs[idx])
            log_records.append({
                "idx": idx,
                "question": question,
                "recall@5": result.recall,
                "hit@5": result.hit,
                "fallback_used": True,
            })
            recalls.append(result.recall)
            hits.append(result.hit)
            continue
        
        mem_data = memory_bank[question]
        buffer_data = buffer[question]
        qc_edges_step1 = buffer_data.get("qc_edges_step1", [])
        intersection_qc_edges = buffer_data.get("intersection_qc_edges", [])
        
        stored_data = mem_data.get("retrieved_triples_with_scores", [])
        
        if stored_data:
            stored_triples = [tuple(item["triple"]) for item in stored_data]
            stored_scores = [item["score"] for item in stored_data]
            
            fact_scores_vec, seed_indices, seed_facts = build_seed_vector_from_stored(
                hippo, stored_triples, stored_scores
            )
            
            # 1. Extra QCEdge 선택
            extra_qc_edges = select_extra_qc_edges_by_percentile(qc_edges_step1, percentile=percentile)
            
            # 2. Intersection + Extra로 그래프 강화
            hippo.graph = strengthen_edges_with_intersection_and_extra(
                original_graph,
                intersection_qc_edges,
                extra_qc_edges,
                theta=config.theta_step3,
                theta_multiplier=theta_multiplier,
                weight_upper_bound=weight_upper_bound
            )
            
            # 3. PPR → Passage
            result = run_graph_search(
                hippo=hippo,
                question=question,
                gold=gold_docs[idx],
                fact_scores_vec=fact_scores_vec,
                seed_indices=seed_indices,
                seed_facts=seed_facts,
            )
            
            log_records.append({
                "idx": idx,
                "question": question,
                "percentile": percentile,
                "theta_multiplier": theta_multiplier,
                "weight_upper_bound": weight_upper_bound,
                "intersection_qc_edges": intersection_qc_edges,
                "extra_qc_edges": extra_qc_edges,
                "retrieved_passages_top5": result.passages,
                "retrieved_scores_top5": result.scores,
                "gold_docs": gold_docs[idx],
                "recall@5": result.recall,
                "hit@5": result.hit,
                "fallback_used": result.fallback_used,
            })
            
            # 그래프 복원
            hippo.graph = original_graph.copy()
        else:
            result = run_fallback_search(hippo, question, gold_docs[idx])
            log_records.append({
                "idx": idx,
                "question": question,
                "recall@5": result.recall,
                "hit@5": result.hit,
                "fallback_used": True,
            })
        
        recalls.append(result.recall)
        hits.append(result.hit)
    
    metrics = {
        "mean_recall@5": float(np.mean(recalls)) if recalls else 0.0,
        "hit_rate@5": float(np.mean(hits)) if hits else 0.0,
    }
    
    return log_records, metrics


print("✓ Step 실행 함수 정의 완료")

✓ Step 실행 함수 정의 완료


In [28]:
# ============================================================
# 12. Main Pipeline Function
# ============================================================
"""
전체 파이프라인 실행 함수
Step 250 → 500 → 750 → 1000 순차 실행
"""

def run_full_pipeline(
    dataset_name: str = None,
    config: PipelineConfig = None,
    query_list: List[str] = None,
    step_to_hippo: Dict[int, HippoRAG] = None,
    verbose: bool = True
) -> pd.DataFrame:
    """
    전체 Continual Learning 파이프라인 실행
    
    Step 250 (초기 검색) → Step 500 (QCEdge 강화) 
    → Step 750 (Intersection+Extra 강화, θ×15, wub=6) 
    → Step 1000 (강한 강화, θ×30, wub=20)
    
    Args:
        config: 파이프라인 설정
        query_list: 특정 쿼리 리스트 (None이면 전체 데이터셋)
        step_to_hippo: 미리 로드한 HippoRAG 객체들 (step -> hippo)
        verbose: 진행 상황 출력 여부
    
    Returns:
        Step별 성능 요약 DataFrame
    """
    config = config or CONFIG
    output_dir = config.get_output_dir(dataset_name)
    os.makedirs(output_dir, exist_ok=True)
    
    # 데이터 로드
    if verbose:
        print("=" * 60)
        print("데이터 로드 중...")
    dataset_name = dataset_name or (CONFIG.dataset_names[0] if CONFIG.dataset_names else "musique")
    samples = load_samples(dataset_name)
    all_queries = get_queries(samples)
    all_gold_docs = get_gold_docs(samples, dataset_name)
    
    # 쿼리 필터링
    if query_list:
        queries, gold_docs = [], []
        lookup = {q: i for i, q in enumerate(all_queries)}
        for query in query_list[:config.max_samples]:
            if query in lookup:
                idx = lookup[query]
                queries.append(query)
                gold_docs.append(all_gold_docs[idx])
    else:
        queries = all_queries[:config.max_samples]
        gold_docs = all_gold_docs[:config.max_samples]
    
    if verbose:
        print(f"  총 {len(queries)}개 쿼리 로드 완료")
    
    summaries = []
    
    # ========================
    # Step 1: Step 250
    # ========================
    memory_bank_path = Path(output_dir) / "memory_bank.json"
    buffer_path = Path(output_dir) / "buffer.json"
    step250_log_path = Path(output_dir) / "step_250_log.json"
    
    if memory_bank_path.exists() and buffer_path.exists() and step250_log_path.exists():
        if verbose:
            print("\n" + "=" * 60)
            print("[Step 250] 캐시된 결과 로드 중...")
        memory_bank = load_json(str(memory_bank_path))
        buffer = load_json(str(buffer_path))
        step250_log = load_json(str(step250_log_path))
        metrics_250 = {
            "mean_recall@5": np.mean([r["recall@5"] for r in step250_log]),
            "hit_rate@5": np.mean([r["hit@5"] for r in step250_log]),
        }
    else:
        if verbose:
            print("\n" + "=" * 60)
            print("[Step 250] 초기 검색 시작...")
        
        hippo_250 = step_to_hippo.get(250) if step_to_hippo else None
        if hippo_250 is None:
            hippo_250 = build_hipporag(250, dataset_name, config)
        
        memory_bank, buffer, step250_log, metrics_250 = run_step1(
            hippo=hippo_250,
            queries=queries,
            gold_docs=gold_docs,
            config=config,
            verbose=verbose
        )
        
        save_json(memory_bank, str(memory_bank_path))
        save_json(buffer, str(buffer_path))
        save_json(step250_log, str(step250_log_path))
    
    summaries.append({"step": 250, **metrics_250})
    if verbose:
        print(f"  → Recall@5: {metrics_250['mean_recall@5']:.4f}, Hit@5: {metrics_250['hit_rate@5']:.4f}")
    
    # ========================
    # Step 2: Step 500
    # ========================
    step500_log_path = Path(output_dir) / "step_500_log.json"
    
    # Check if intersection_qc_edges exists in buffer
    has_intersection = any(
        isinstance(v, dict) and v.get("intersection_qc_edges")
        for v in buffer.values()
    )
    
    if step500_log_path.exists() and has_intersection:
        if verbose:
            print("\n" + "=" * 60)
            print("[Step 500] 캐시된 결과 로드 중...")
        step500_log = load_json(str(step500_log_path))
        metrics_500 = {
            "mean_recall@5": np.mean([r["recall@5"] for r in step500_log]),
            "hit_rate@5": np.mean([r["hit@5"] for r in step500_log]),
        }
        buffer = load_json(str(buffer_path))  # Reload with intersection data
    else:
        if verbose:
            print("\n" + "=" * 60)
            print("[Step 500] QCEdge 강화 검색 시작...")
            print(f"  θ={config.theta_step2}, weight_upper_bound={config.wub_step2}")
        
        hippo_500 = step_to_hippo.get(500) if step_to_hippo else None
        if hippo_500 is None:
            hippo_500 = build_hipporag(500, dataset_name, config)
        
        buffer, step500_log, metrics_500 = run_step2(
            hippo=hippo_500,
            queries=queries,
            gold_docs=gold_docs,
            memory_bank=memory_bank,
            buffer=buffer,
            config=config,
            verbose=verbose
        )
        
        save_json(buffer, str(buffer_path))
        save_json(step500_log, str(step500_log_path))
    
    summaries.append({"step": 500, **metrics_500})
    if verbose:
        print(f"  → Recall@5: {metrics_500['mean_recall@5']:.4f}, Hit@5: {metrics_500['hit_rate@5']:.4f}")
    
    # ========================
    # Step 3: Step 750
    # ========================
    step750_log_path = Path(output_dir) / "step_750_log.json"
    
    if step750_log_path.exists():
        if verbose:
            print("\n" + "=" * 60)
            print("[Step 750] 캐시된 결과 로드 중...")
        step750_log = load_json(str(step750_log_path))
        metrics_750 = {
            "mean_recall@5": np.mean([r["recall@5"] for r in step750_log]),
            "hit_rate@5": np.mean([r["hit@5"] for r in step750_log]),
        }
    else:
        if verbose:
            print("\n" + "=" * 60)
            print("[Step 750] Intersection + Extra QCEdge 강화 검색 시작...")
            print(f"  θ_multiplier={config.theta_mult_step3}, weight_upper_bound={config.wub_step3}")
        
        hippo_750 = step_to_hippo.get(750) if step_to_hippo else None
        if hippo_750 is None:
            hippo_750 = build_hipporag(750, dataset_name, config)
        
        step750_log, metrics_750 = run_step3_or_4(
            hippo=hippo_750,
            queries=queries,
            gold_docs=gold_docs,
            memory_bank=memory_bank,
            buffer=buffer,
            step_num=750,
            theta_multiplier=config.theta_mult_step3,
            weight_upper_bound=config.wub_step3,
            percentile=config.percentile_step3,
            config=config,
            verbose=verbose
        )
        
        save_json(step750_log, str(step750_log_path))
    
    summaries.append({"step": 750, **metrics_750})
    if verbose:
        print(f"  → Recall@5: {metrics_750['mean_recall@5']:.4f}, Hit@5: {metrics_750['hit_rate@5']:.4f}")
    
    # ========================
    # Step 4: Step 1000
    # ========================
    step1000_log_path = Path(output_dir) / "step_1000_log.json"
    
    if step1000_log_path.exists():
        if verbose:
            print("\n" + "=" * 60)
            print("[Step 1000] 캐시된 결과 로드 중...")
        step1000_log = load_json(str(step1000_log_path))
        metrics_1000 = {
            "mean_recall@5": np.mean([r["recall@5"] for r in step1000_log]),
            "hit_rate@5": np.mean([r["hit@5"] for r in step1000_log]),
        }
    else:
        if verbose:
            print("\n" + "=" * 60)
            print("[Step 1000] 강한 Intersection + Extra QCEdge 강화 검색 시작...")
            print(f"  θ_multiplier={config.theta_mult_step4}, weight_upper_bound={config.wub_step4}")
        
        hippo_1000 = step_to_hippo.get(1000) if step_to_hippo else None
        if hippo_1000 is None:
            hippo_1000 = build_hipporag(1000, dataset_name, config)
        
        step1000_log, metrics_1000 = run_step3_or_4(
            hippo=hippo_1000,
            queries=queries,
            gold_docs=gold_docs,
            memory_bank=memory_bank,
            buffer=buffer,
            step_num=1000,
            theta_multiplier=config.theta_mult_step4,
            weight_upper_bound=config.wub_step4,
            percentile=config.percentile_step4,
            config=config,
            verbose=verbose
        )
        
        save_json(step1000_log, str(step1000_log_path))
    
    summaries.append({"step": 1000, **metrics_1000})
    if verbose:
        print(f"  → Recall@5: {metrics_1000['mean_recall@5']:.4f}, Hit@5: {metrics_1000['hit_rate@5']:.4f}")
    
    # 결과 요약
    result_df = pd.DataFrame(summaries).sort_values("step").reset_index(drop=True)
    
    if verbose:
        print("\n" + "=" * 60)
        print("전체 파이프라인 완료!")
        print("=" * 60)
        print(result_df.to_string(index=False))
    
    return result_df


print("✓ 메인 파이프라인 함수 정의 완료")

✓ 메인 파이프라인 함수 정의 완료


# 데이터셋 실험 실행 방법

In [31]:
# # 예제 1: 단일 데이터셋 실험 (musique)
# CONFIG.dataset_names = ["musique"]
# results_df1 = run_full_pipeline(
#     dataset_name="musique",
#     config=CONFIG,
#     verbose=True
# )

# 예제 2: 단일 데이터셋 실험 (hotpotqa)
CONFIG.dataset_names = ["hotpotqa"]
results_df = run_full_pipeline(
    dataset_name="hotpotqa",
    config=CONFIG,
    verbose=True
)

# # 예제 3: 단일 데이터셋 실험 (2wikimultihopqa)
# CONFIG.dataset_names = ["2wikimultihopqa"]
# results_df2 = run_full_pipeline(
#     dataset_name="2wikimultihopqa",
#     config=CONFIG,
#     verbose=True
# )

# # 예제 4: 여러 데이터셋 자동 실험
# CONFIG.dataset_names = ["musique", "hotpotqa", "2wikimultihopqa"]
# results = run_experiments_for_datasets(
#     dataset_names=["musique", "hotpotqa", "2wikimultihopqa"],
#     config=CONFIG,
#     verbose=True
# )

데이터 로드 중...
  총 250개 쿼리 로드 완료

[Step 250] 캐시된 결과 로드 중...
  → Recall@5: 0.9480, Hit@5: 0.9920

[Step 500] QCEdge 강화 검색 시작...
  θ=15.0, weight_upper_bound=3.0


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [29]:
results_df1.to_csv('musique_CL_QD.csv', index=False)
results_df2.to_csv('2wikimultihopqa_CL_QD.csv', index=False)